Taking a probabilistic approach to deep learning allows to account for uncertainty, so that models can assign less levels of confidence to incorrect predictions. Sources of uncertainty can be found in the data, due to measurement error or noise in the labels, or the model, due to insufficient data availability for the model to learn effectively.

This example demonstrates how to build basic probabilistic Bayesian neural networks to account for these two types of uncertainty. We use TensorFlow Probability library, which is compatible with Keras API.

In [1]:
!pip install tensorflow-datasets

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=8c6b251f89ecd2c671b2f08f0d4ff8be048a1c01ae52112c4db82f5bdd6e1940
  Stored in directory: c:\users\mfund\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.13.0
    Uninstalling absl-py-0.13.0:
      Successfully uninstalled absl-py-0.13.0


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

# Creating training & evaluation datasets

Here, we load the wine_quality dataset using tfds.load(), and we convert the target feature to float. Then, we shuffle the dataset and split it into training and test sets. We take the first train_size examples as the train split, and the rest as the test split.

In [10]:
def train_and_test_splits(train_size, batch_size = 1):
    
    dataset = (
    tfds.load(name='wine_quality', as_supervised = True, split='train')
    .map(lambda x,y: (x,tf.cast(y,tf.float32)))
    .prefetch(buffer_size=dataset_size)
    .cache()
    )
    #shuffle with a buffer the same sie as the dst.
    
    train_dataset = (
    dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)
    
    return train_dataset, test_dataset

In [20]:
hidden_units = [8,8]
learning_rate = 0.001

def run_experiment(model, loss, train_dataset, test_dataset):
    model.compile(
    optimizer = keras.optimizers.RMSprop(learning_rate = learning_rate),
    loss = loss,
    metrics = [keras.metrics.RootMeanSquaredError()],
    )
    
    print('Start training the model...')
    
    model.fit(train_dataset, epochs = num_epochs, validation_data = test_dataset)
    print('Model training finished...')
    _,rmse = model.evaluate(train_dataset, verbose=0)
    print(f'Trains RMSE: {round(rmse,3)}')
    
    print('evaluating model perfomance... ')
    
    _,rmse = model.evaluate(test_dataset, verbose=0)
    print(f'Test RMSE: {round(rmse,3)}')


In [21]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]

def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
        name = feature_name, shape=(1,), dtype = tf.float32
        )
    return inputs

In [22]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [11]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = train_and_test_splits(train_size, batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling wine_quality-train.tfrecord...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Dataset wine_quality downloaded and prepared to C:\Users\Mfund\tensorflow_datasets\wine_quality\white\1.0.0. Subsequent calls will reuse this data.


In [23]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 1s 16ms/step - loss: 33.7753 - root_mean_squared_error: 5.8117 - val_loss: 30.9093 - val_root_mean_squared_error: 5.5596
Epoch 2/100
17/17 [==============================] - 0s 9ms/step - loss: 31.7175 - root_mean_squared_error: 5.6318 - val_loss: 28.9745 - val_root_mean_squared_error: 5.3828
Epoch 3/100
17/17 [==============================] - 0s 10ms/step - loss: 29.9789 - root_mean_squared_error: 5.4753 - val_loss: 27.2409 - val_root_mean_squared_error: 5.2193
Epoch 4/100
17/17 [==============================] - 0s 11ms/step - loss: 28.3169 - root_mean_squared_error: 5.3214 - val_loss: 25.5843 - val_root_mean_squared_error: 5.0581
Epoch 5/100
17/17 [==============================] - 0s 11ms/step - loss: 26.7032 - root_mean_squared_error: 5.1675 - val_loss: 23.9995 - val_root_mean_squared_error: 4.8989
Epoch 6/100
17/17 [==============================] - 0s 9ms/step - loss: 25.1416 - root_mean_squared_er

Epoch 96/100
17/17 [==============================] - 0s 7ms/step - loss: 0.5631 - root_mean_squared_error: 0.7504 - val_loss: 0.5417 - val_root_mean_squared_error: 0.7360
Epoch 97/100
17/17 [==============================] - 0s 7ms/step - loss: 0.5623 - root_mean_squared_error: 0.7499 - val_loss: 0.5403 - val_root_mean_squared_error: 0.7351
Epoch 98/100
17/17 [==============================] - 0s 7ms/step - loss: 0.5639 - root_mean_squared_error: 0.7509 - val_loss: 0.5398 - val_root_mean_squared_error: 0.7347
Epoch 99/100
17/17 [==============================] - 0s 7ms/step - loss: 0.5618 - root_mean_squared_error: 0.7495 - val_loss: 0.5388 - val_root_mean_squared_error: 0.7340
Epoch 100/100
17/17 [==============================] - 0s 7ms/step - loss: 0.5611 - root_mean_squared_error: 0.7490 - val_loss: 0.5380 - val_root_mean_squared_error: 0.7335
Model training finished...
Trains RMSE: 0.747
evaluating model perfomance... 
Test RMSE: 0.733


We take a sample from the test set use the model to obtain predictions for them. Note that since the baseline model is deterministic, we get a single a point estimate prediction for each test example, with no information about the uncertainty of the model nor the prediction.

In [24]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 6.1 - Actual: 6.0
Predicted: 6.2 - Actual: 6.0
Predicted: 6.6 - Actual: 7.0
Predicted: 5.0 - Actual: 5.0
Predicted: 5.6 - Actual: 5.0
Predicted: 6.0 - Actual: 6.0
Predicted: 5.6 - Actual: 6.0
Predicted: 5.9 - Actual: 7.0
Predicted: 5.7 - Actual: 8.0
Predicted: 5.7 - Actual: 5.0


# Bayesian neural network

The object of the Bayesian approach for modeling neural networks is to capture the epistemic uncertainty, which is uncertainty about the model fitness, due to limited training data.

The idea is that, instead of learning specific weight (and bias) values in the neural network, the Bayesian approach learns weight distributions - from which we can sample to produce an output for a given input - to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights, and the training process is to learn the parameters of these distributions.


In [25]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [26]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

The epistemic uncertainty can be reduced as we increase the size of the training data. That is, the more data the BNN model sees, the more it is certain about its estimates for the weights (distribution parameters). Let's test this behaviour by training the BNN model on a small subset of the training set, and then on the full training set, to compare the output variances

In [27]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 195ms/step - loss: 34.3439 - root_mean_squared_error: 5.8594 - val_loss: 30.4666 - val_root_mean_squared_error: 5.5185
Epoch 2/500
5/5 [==============================] - 0s 22ms/step - loss: 33.2786 - root_mean_squared_error: 5.7675 - val_loss: 36.3452 - val_root_mean_squared_error: 6.0278
Epoch 3/500
5/5 [==============================] - 0s 24ms/step - loss: 32.0317 - root_mean_squared_error: 5.6583 - val_loss: 29.6401 - val_root_mean_squared_error: 5.4431
Epoch 4/500
5/5 [==============================] - 0s 24ms/step - loss: 31.9190 - root_mean_squared_error: 5.6486 - val_loss: 31.1966 - val_root_mean_squared_error: 5.5837
Epoch 5/500
5/5 [==============================] - 0s 24ms/step - loss: 30.6819 - root_mean_squared_error: 5.5378 - val_loss: 34.7862 - val_root_mean_squared_error: 5.8966
Epoch 6/500
5/5 [==============================] - 0s 24ms/step - loss: 33.7452 - root_mean_squared_error: 5.80

Epoch 96/500
5/5 [==============================] - 0s 18ms/step - loss: 6.1786 - root_mean_squared_error: 2.4812 - val_loss: 5.4827 - val_root_mean_squared_error: 2.3368
Epoch 97/500
5/5 [==============================] - 0s 16ms/step - loss: 4.5649 - root_mean_squared_error: 2.1307 - val_loss: 5.4804 - val_root_mean_squared_error: 2.3364
Epoch 98/500
5/5 [==============================] - 0s 16ms/step - loss: 5.8266 - root_mean_squared_error: 2.4091 - val_loss: 4.2238 - val_root_mean_squared_error: 2.0501
Epoch 99/500
5/5 [==============================] - 0s 33ms/step - loss: 5.4041 - root_mean_squared_error: 2.3202 - val_loss: 4.0273 - val_root_mean_squared_error: 2.0016
Epoch 100/500
5/5 [==============================] - 0s 28ms/step - loss: 5.8756 - root_mean_squared_error: 2.4193 - val_loss: 4.0880 - val_root_mean_squared_error: 2.0158
Epoch 101/500
5/5 [==============================] - 0s 18ms/step - loss: 5.2740 - root_mean_squared_error: 2.2921 - val_loss: 4.1034 - val_root

Epoch 144/500
5/5 [==============================] - 0s 18ms/step - loss: 1.0556 - root_mean_squared_error: 1.0128 - val_loss: 0.9115 - val_root_mean_squared_error: 0.9386
Epoch 145/500
5/5 [==============================] - 0s 16ms/step - loss: 1.2700 - root_mean_squared_error: 1.1162 - val_loss: 1.2457 - val_root_mean_squared_error: 1.1046
Epoch 146/500
5/5 [==============================] - 0s 16ms/step - loss: 1.0220 - root_mean_squared_error: 0.9972 - val_loss: 0.8816 - val_root_mean_squared_error: 0.9249
Epoch 147/500
5/5 [==============================] - 0s 18ms/step - loss: 0.8462 - root_mean_squared_error: 0.9033 - val_loss: 1.0039 - val_root_mean_squared_error: 0.9878
Epoch 148/500
5/5 [==============================] - 0s 16ms/step - loss: 1.1343 - root_mean_squared_error: 1.0520 - val_loss: 0.8840 - val_root_mean_squared_error: 0.9251
Epoch 149/500
5/5 [==============================] - 0s 18ms/step - loss: 0.8072 - root_mean_squared_error: 0.8816 - val_loss: 0.9915 - val_

Epoch 239/500
5/5 [==============================] - 0s 16ms/step - loss: 0.7546 - root_mean_squared_error: 0.8531 - val_loss: 0.8065 - val_root_mean_squared_error: 0.8835
Epoch 240/500
5/5 [==============================] - 0s 16ms/step - loss: 0.7427 - root_mean_squared_error: 0.8462 - val_loss: 0.7087 - val_root_mean_squared_error: 0.8253
Epoch 241/500
5/5 [==============================] - 0s 18ms/step - loss: 0.7633 - root_mean_squared_error: 0.8572 - val_loss: 0.7881 - val_root_mean_squared_error: 0.8732
Epoch 242/500
5/5 [==============================] - 0s 18ms/step - loss: 0.7279 - root_mean_squared_error: 0.8384 - val_loss: 0.7623 - val_root_mean_squared_error: 0.8571
Epoch 243/500
5/5 [==============================] - 0s 20ms/step - loss: 0.7566 - root_mean_squared_error: 0.8539 - val_loss: 0.7477 - val_root_mean_squared_error: 0.8502
Epoch 244/500
5/5 [==============================] - 0s 16ms/step - loss: 0.7649 - root_mean_squared_error: 0.8578 - val_loss: 0.7493 - val_

5/5 [==============================] - 0s 16ms/step - loss: 0.7045 - root_mean_squared_error: 0.8233 - val_loss: 0.6783 - val_root_mean_squared_error: 0.8079
Epoch 334/500
5/5 [==============================] - 0s 16ms/step - loss: 0.6342 - root_mean_squared_error: 0.7793 - val_loss: 0.7702 - val_root_mean_squared_error: 0.8639
Epoch 335/500
5/5 [==============================] - 0s 16ms/step - loss: 0.7556 - root_mean_squared_error: 0.8545 - val_loss: 0.6684 - val_root_mean_squared_error: 0.7989
Epoch 336/500
5/5 [==============================] - 0s 16ms/step - loss: 0.7208 - root_mean_squared_error: 0.8315 - val_loss: 0.7328 - val_root_mean_squared_error: 0.8430
Epoch 337/500
5/5 [==============================] - 0s 16ms/step - loss: 0.7133 - root_mean_squared_error: 0.8294 - val_loss: 0.7061 - val_root_mean_squared_error: 0.8242
Epoch 338/500
5/5 [==============================] - 0s 14ms/step - loss: 0.6819 - root_mean_squared_error: 0.8100 - val_loss: 0.6605 - val_root_mean_squa

Epoch 428/500
5/5 [==============================] - 0s 18ms/step - loss: 0.6452 - root_mean_squared_error: 0.7874 - val_loss: 0.6315 - val_root_mean_squared_error: 0.7791
Epoch 429/500
5/5 [==============================] - 0s 16ms/step - loss: 0.6373 - root_mean_squared_error: 0.7806 - val_loss: 0.6362 - val_root_mean_squared_error: 0.7812
Epoch 430/500
5/5 [==============================] - 0s 16ms/step - loss: 0.6109 - root_mean_squared_error: 0.7654 - val_loss: 0.6440 - val_root_mean_squared_error: 0.7874
Epoch 431/500
5/5 [==============================] - 0s 16ms/step - loss: 0.6256 - root_mean_squared_error: 0.7717 - val_loss: 0.6936 - val_root_mean_squared_error: 0.8159
Epoch 432/500
5/5 [==============================] - 0s 16ms/step - loss: 0.6643 - root_mean_squared_error: 0.7978 - val_loss: 0.6898 - val_root_mean_squared_error: 0.8157
Epoch 433/500
5/5 [==============================] - 0s 16ms/step - loss: 0.6655 - root_mean_squared_error: 0.7984 - val_loss: 0.6742 - val_

Since we have trained a BNN model, the model produces a different output each time we call it with the same input, since each time a new set of weights are sampled from the distributions to construct the network and produce an output. The less certain the mode weights are, the more variability (wider range) we will see in the outputs of the same inputs.

In [28]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 6.07, min: 5.5, max: 6.43, range: 0.94 - Actual: 6.0
Predictions mean: 6.23, min: 5.7, max: 6.44, range: 0.74 - Actual: 6.0
Predictions mean: 6.33, min: 6.08, max: 6.52, range: 0.44 - Actual: 7.0
Predictions mean: 5.09, min: 4.26, max: 5.76, range: 1.5 - Actual: 5.0
Predictions mean: 5.87, min: 5.06, max: 6.23, range: 1.17 - Actual: 5.0
Predictions mean: 5.94, min: 5.43, max: 6.32, range: 0.89 - Actual: 6.0
Predictions mean: 5.82, min: 5.16, max: 6.24, range: 1.07 - Actual: 6.0
Predictions mean: 5.98, min: 5.36, max: 6.36, range: 1.0 - Actual: 7.0
Predictions mean: 5.71, min: 4.72, max: 6.24, range: 1.52 - Actual: 8.0
Predictions mean: 5.86, min: 5.03, max: 6.37, range: 1.35 - Actual: 5.0


In [29]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 3s 56ms/step - loss: 16.3730 - root_mean_squared_error: 4.0459 - val_loss: 14.3438 - val_root_mean_squared_error: 3.7868
Epoch 2/500
17/17 [==============================] - 0s 7ms/step - loss: 14.1643 - root_mean_squared_error: 3.7629 - val_loss: 13.6934 - val_root_mean_squared_error: 3.7000
Epoch 3/500
17/17 [==============================] - 0s 7ms/step - loss: 12.8462 - root_mean_squared_error: 3.5835 - val_loss: 13.4235 - val_root_mean_squared_error: 3.6634
Epoch 4/500
17/17 [==============================] - 0s 9ms/step - loss: 13.7804 - root_mean_squared_error: 3.7117 - val_loss: 11.8211 - val_root_mean_squared_error: 3.4374
Epoch 5/500
17/17 [==============================] - 0s 11ms/step - loss: 10.8431 - root_mean_squared_error: 3.2922 - val_loss: 13.0553 - val_root_mean_squared_error: 3.6125
Epoch 6/500
17/17 [==============================] - 0s 9ms/step - loss: 11.3089 - root_mean_squared_erro

17/17 [==============================] - 0s 8ms/step - loss: 0.7426 - root_mean_squared_error: 0.8574 - val_loss: 0.7290 - val_root_mean_squared_error: 0.8493
Epoch 49/500
17/17 [==============================] - 0s 8ms/step - loss: 0.7653 - root_mean_squared_error: 0.8708 - val_loss: 0.7667 - val_root_mean_squared_error: 0.8716
Epoch 50/500
17/17 [==============================] - 0s 9ms/step - loss: 0.7907 - root_mean_squared_error: 0.8849 - val_loss: 0.7480 - val_root_mean_squared_error: 0.8607
Epoch 51/500
17/17 [==============================] - 0s 9ms/step - loss: 0.7387 - root_mean_squared_error: 0.8552 - val_loss: 0.7072 - val_root_mean_squared_error: 0.8365
Epoch 52/500
17/17 [==============================] - 0s 9ms/step - loss: 0.7493 - root_mean_squared_error: 0.8609 - val_loss: 0.7202 - val_root_mean_squared_error: 0.8441
Epoch 53/500
17/17 [==============================] - 0s 8ms/step - loss: 0.7526 - root_mean_squared_error: 0.8632 - val_loss: 0.7292 - val_root_mean_squ

Epoch 96/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6752 - root_mean_squared_error: 0.8154 - val_loss: 0.6696 - val_root_mean_squared_error: 0.8123
Epoch 97/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6715 - root_mean_squared_error: 0.8133 - val_loss: 0.6436 - val_root_mean_squared_error: 0.7950
Epoch 98/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6580 - root_mean_squared_error: 0.8047 - val_loss: 0.6613 - val_root_mean_squared_error: 0.8071
Epoch 99/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6512 - root_mean_squared_error: 0.8009 - val_loss: 0.6458 - val_root_mean_squared_error: 0.7979
Epoch 100/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6558 - root_mean_squared_error: 0.8039 - val_loss: 0.6233 - val_root_mean_squared_error: 0.7823
Epoch 101/500
17/17 [==============================] - 0s 9ms/step - loss: 0.6611 - root_mean_squared_error: 0.8066 - val_loss: 0.6170 - va

17/17 [==============================] - 0s 6ms/step - loss: 0.6398 - root_mean_squared_error: 0.7919 - val_loss: 0.6165 - val_root_mean_squared_error: 0.7777
Epoch 191/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6339 - root_mean_squared_error: 0.7887 - val_loss: 0.6297 - val_root_mean_squared_error: 0.7858
Epoch 192/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6150 - root_mean_squared_error: 0.7761 - val_loss: 0.6011 - val_root_mean_squared_error: 0.7674
Epoch 193/500
17/17 [==============================] - 0s 5ms/step - loss: 0.6228 - root_mean_squared_error: 0.7814 - val_loss: 0.6432 - val_root_mean_squared_error: 0.7945
Epoch 194/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6162 - root_mean_squared_error: 0.7768 - val_loss: 0.6043 - val_root_mean_squared_error: 0.7691
Epoch 195/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6254 - root_mean_squared_error: 0.7833 - val_loss: 0.6154 - val_root_mea

17/17 [==============================] - 0s 12ms/step - loss: 0.6077 - root_mean_squared_error: 0.7708 - val_loss: 0.6166 - val_root_mean_squared_error: 0.7774
Epoch 285/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6073 - root_mean_squared_error: 0.7708 - val_loss: 0.6089 - val_root_mean_squared_error: 0.7716
Epoch 286/500
17/17 [==============================] - 0s 9ms/step - loss: 0.6069 - root_mean_squared_error: 0.7709 - val_loss: 0.5960 - val_root_mean_squared_error: 0.7627
Epoch 287/500
17/17 [==============================] - 0s 9ms/step - loss: 0.6139 - root_mean_squared_error: 0.7749 - val_loss: 0.5992 - val_root_mean_squared_error: 0.7652
Epoch 288/500
17/17 [==============================] - 0s 7ms/step - loss: 0.6168 - root_mean_squared_error: 0.7769 - val_loss: 0.6059 - val_root_mean_squared_error: 0.7700
Epoch 289/500
17/17 [==============================] - 0s 8ms/step - loss: 0.6155 - root_mean_squared_error: 0.7760 - val_loss: 0.6023 - val_root_m

17/17 [==============================] - 0s 10ms/step - loss: 0.6030 - root_mean_squared_error: 0.7674 - val_loss: 0.5895 - val_root_mean_squared_error: 0.7589
Epoch 379/500
17/17 [==============================] - 0s 9ms/step - loss: 0.6061 - root_mean_squared_error: 0.7689 - val_loss: 0.5888 - val_root_mean_squared_error: 0.7559
Epoch 380/500
17/17 [==============================] - 0s 10ms/step - loss: 0.6026 - root_mean_squared_error: 0.7668 - val_loss: 0.6028 - val_root_mean_squared_error: 0.7668
Epoch 381/500
17/17 [==============================] - 0s 6ms/step - loss: 0.5998 - root_mean_squared_error: 0.7652 - val_loss: 0.5847 - val_root_mean_squared_error: 0.7552
Epoch 382/500
17/17 [==============================] - 0s 6ms/step - loss: 0.6005 - root_mean_squared_error: 0.7659 - val_loss: 0.6430 - val_root_mean_squared_error: 0.7932
Epoch 383/500
17/17 [==============================] - 0s 6ms/step - loss: 0.5989 - root_mean_squared_error: 0.7651 - val_loss: 0.6078 - val_root_m

17/17 [==============================] - 0s 12ms/step - loss: 0.5984 - root_mean_squared_error: 0.7641 - val_loss: 0.5867 - val_root_mean_squared_error: 0.7563
Epoch 473/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6179 - root_mean_squared_error: 0.7767 - val_loss: 0.5968 - val_root_mean_squared_error: 0.7630
Epoch 474/500
17/17 [==============================] - 0s 12ms/step - loss: 0.5959 - root_mean_squared_error: 0.7624 - val_loss: 0.6198 - val_root_mean_squared_error: 0.7780
Epoch 475/500
17/17 [==============================] - 0s 11ms/step - loss: 0.6070 - root_mean_squared_error: 0.7695 - val_loss: 0.5971 - val_root_mean_squared_error: 0.7633
Epoch 476/500
17/17 [==============================] - 0s 12ms/step - loss: 0.6010 - root_mean_squared_error: 0.7652 - val_loss: 0.6114 - val_root_mean_squared_error: 0.7723
Epoch 477/500
17/17 [==============================] - ETA: 0s - loss: 0.6001 - root_mean_squared_error: 0.764 - 0s 12ms/step - loss: 0.6012 - r

Notice that the model trained with the full training dataset shows smaller range (uncertainty) in the prediction values for the same inputs, compared to the model trained with a subset of the training dataset.

# Probabilistic Bayesian neural network

So far, the output of the standard and the Bayesian NN models that we built is deterministic, that is, produces a point estimate as a prediction for a given example. We can create a probabilistic NN by letting the model output a distribution. In this case, the model captures the aleatoric uncertainty as well, which is due to irreducible noise in the data, or to the stochastic nature of the process generating the data.

In this example, we model the output as a IndependentNormal distribution, with learnable mean and variance parameters. If the task was classification, we would have used IndependentBernoulli with binary classes, and OneHotCategorical with multiple classes, to model distribution of the model output.


In [30]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

Since the output of the model is a distribution, rather than a point estimate, we use the negative loglikelihood as our loss function to compute how likely to see the true data (targets) from the estimated distribution produced by the model.

In [31]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 3s 52ms/step - loss: 40.3108 - root_mean_squared_error: 5.6437 - val_loss: 26.8973 - val_root_mean_squared_error: 5.4501
Epoch 2/1000
17/17 [==============================] - 0s 9ms/step - loss: 33.2217 - root_mean_squared_error: 5.5741 - val_loss: 23.1883 - val_root_mean_squared_error: 5.0316
Epoch 3/1000
17/17 [==============================] - 0s 9ms/step - loss: 30.9093 - root_mean_squared_error: 5.5205 - val_loss: 34.5898 - val_root_mean_squared_error: 5.7441
Epoch 4/1000
17/17 [==============================] - 0s 11ms/step - loss: 30.1454 - root_mean_squared_error: 5.5465 - val_loss: 33.0665 - val_root_mean_squared_error: 5.2805
Epoch 5/1000
17/17 [==============================] - 0s 9ms/step - loss: 21.8021 - root_mean_squared_error: 5.3697 - val_loss: 26.0577 - val_root_mean_squared_error: 5.2085
Epoch 6/1000
17/17 [==============================] - 0s 10ms/step - loss: 16.1870 - root_mean_squar

Epoch 48/1000
17/17 [==============================] - 0s 9ms/step - loss: 1.4946 - root_mean_squared_error: 1.7242 - val_loss: 1.4410 - val_root_mean_squared_error: 1.6167
Epoch 49/1000
17/17 [==============================] - 0s 10ms/step - loss: 1.4495 - root_mean_squared_error: 1.6201 - val_loss: 1.4037 - val_root_mean_squared_error: 1.5214
Epoch 50/1000
17/17 [==============================] - 0s 10ms/step - loss: 1.4337 - root_mean_squared_error: 1.6059 - val_loss: 1.3777 - val_root_mean_squared_error: 1.5125
Epoch 51/1000
17/17 [==============================] - 0s 9ms/step - loss: 1.3700 - root_mean_squared_error: 1.4718 - val_loss: 1.4588 - val_root_mean_squared_error: 1.7028
Epoch 52/1000
17/17 [==============================] - 0s 12ms/step - loss: 1.3897 - root_mean_squared_error: 1.4471 - val_loss: 1.3596 - val_root_mean_squared_error: 1.3942
Epoch 53/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.3332 - root_mean_squared_error: 1.3800 - val_loss: 1.32

17/17 [==============================] - 0s 6ms/step - loss: 1.1924 - root_mean_squared_error: 1.1046 - val_loss: 1.1963 - val_root_mean_squared_error: 1.1233
Epoch 143/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1952 - root_mean_squared_error: 1.0955 - val_loss: 1.1823 - val_root_mean_squared_error: 1.0785
Epoch 144/1000
17/17 [==============================] - 0s 9ms/step - loss: 1.2047 - root_mean_squared_error: 1.0998 - val_loss: 1.1781 - val_root_mean_squared_error: 1.1130
Epoch 145/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1941 - root_mean_squared_error: 1.1419 - val_loss: 1.1971 - val_root_mean_squared_error: 1.0957
Epoch 146/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1856 - root_mean_squared_error: 1.1009 - val_loss: 1.1792 - val_root_mean_squared_error: 1.1135
Epoch 147/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1897 - root_mean_squared_error: 1.1092 - val_loss: 1.1928 - val_roo

17/17 [==============================] - 0s 7ms/step - loss: 1.1890 - root_mean_squared_error: 1.1253 - val_loss: 1.1759 - val_root_mean_squared_error: 1.1142
Epoch 190/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1842 - root_mean_squared_error: 1.1089 - val_loss: 1.1746 - val_root_mean_squared_error: 1.1751
Epoch 191/1000
17/17 [==============================] - 0s 5ms/step - loss: 1.1773 - root_mean_squared_error: 1.0981 - val_loss: 1.2001 - val_root_mean_squared_error: 1.1152
Epoch 192/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1825 - root_mean_squared_error: 1.1264 - val_loss: 1.1687 - val_root_mean_squared_error: 1.1371
Epoch 193/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1852 - root_mean_squared_error: 1.1285 - val_loss: 1.1663 - val_root_mean_squared_error: 1.1204
Epoch 194/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1872 - root_mean_squared_error: 1.1034 - val_loss: 1.1572 - val_roo

Epoch 283/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1698 - root_mean_squared_error: 1.1134 - val_loss: 1.2212 - val_root_mean_squared_error: 1.1446
Epoch 284/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1760 - root_mean_squared_error: 1.0845 - val_loss: 1.1611 - val_root_mean_squared_error: 1.1040
Epoch 285/1000
17/17 [==============================] - 0s 9ms/step - loss: 1.1696 - root_mean_squared_error: 1.0846 - val_loss: 1.1601 - val_root_mean_squared_error: 1.0734
Epoch 286/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1739 - root_mean_squared_error: 1.1038 - val_loss: 1.1682 - val_root_mean_squared_error: 1.1664
Epoch 287/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1820 - root_mean_squared_error: 1.1214 - val_loss: 1.1972 - val_root_mean_squared_error: 1.1067
Epoch 288/1000
17/17 [==============================] - 0s 8ms/step - loss: 1.1681 - root_mean_squared_error: 1.1057 - val_loss: 1

...

17/17 [==============================] - 0s 11ms/step - loss: 1.1603 - root_mean_squared_error: 1.0817 - val_loss: 1.1599 - val_root_mean_squared_error: 1.0894
Epoch 660/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1707 - root_mean_squared_error: 1.0785 - val_loss: 1.1674 - val_root_mean_squared_error: 1.1278
Epoch 661/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1699 - root_mean_squared_error: 1.0869 - val_loss: 1.1655 - val_root_mean_squared_error: 1.0949
Epoch 662/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1654 - root_mean_squared_error: 1.0991 - val_loss: 1.1546 - val_root_mean_squared_error: 1.0929
Epoch 663/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1597 - root_mean_squared_error: 1.0588 - val_loss: 1.1727 - val_root_mean_squared_error: 1.1265
Epoch 664/1000
17/17 [==============================] - 0s 9ms/step - loss: 1.1602 - root_mean_squared_error: 1.0664 - val_loss: 1.1686 - val_r

17/17 [==============================] - 0s 6ms/step - loss: 1.1566 - root_mean_squared_error: 1.0868 - val_loss: 1.1540 - val_root_mean_squared_error: 1.0549
Epoch 707/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1570 - root_mean_squared_error: 1.0765 - val_loss: 1.1499 - val_root_mean_squared_error: 1.1072
Epoch 708/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1578 - root_mean_squared_error: 1.0494 - val_loss: 1.1630 - val_root_mean_squared_error: 1.0513
Epoch 709/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1570 - root_mean_squared_error: 1.0869 - val_loss: 1.1587 - val_root_mean_squared_error: 1.1020
Epoch 710/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1576 - root_mean_squared_error: 1.0465 - val_loss: 1.1522 - val_root_mean_squared_error: 1.0942
Epoch 711/1000
17/17 [==============================] - 0s 6ms/step - loss: 1.1617 - root_mean_squared_error: 1.0673 - val_loss: 1.1649 - val_roo

17/17 [==============================] - 0s 11ms/step - loss: 1.1526 - root_mean_squared_error: 1.0935 - val_loss: 1.1521 - val_root_mean_squared_error: 1.0337
Epoch 801/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1498 - root_mean_squared_error: 1.0642 - val_loss: 1.1595 - val_root_mean_squared_error: 1.0295
Epoch 802/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1601 - root_mean_squared_error: 1.0777 - val_loss: 1.2099 - val_root_mean_squared_error: 1.1104
Epoch 803/1000
17/17 [==============================] - 0s 9ms/step - loss: 1.1495 - root_mean_squared_error: 1.0648 - val_loss: 1.1723 - val_root_mean_squared_error: 1.0762
Epoch 804/1000
17/17 [==============================] - 0s 10ms/step - loss: 1.1570 - root_mean_squared_error: 1.0621 - val_loss: 1.1449 - val_root_mean_squared_error: 1.0468
Epoch 805/1000
17/17 [==============================] - 0s 10ms/step - loss: 1.1578 - root_mean_squared_error: 1.0865 - val_loss: 1.1506 - val_

17/17 [==============================] - 0s 10ms/step - loss: 1.1495 - root_mean_squared_error: 1.0905 - val_loss: 1.1732 - val_root_mean_squared_error: 1.0729
Epoch 895/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1530 - root_mean_squared_error: 1.0729 - val_loss: 1.1553 - val_root_mean_squared_error: 1.0642
Epoch 896/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1562 - root_mean_squared_error: 1.0546 - val_loss: 1.1532 - val_root_mean_squared_error: 1.0803
Epoch 897/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1524 - root_mean_squared_error: 1.0503 - val_loss: 1.1409 - val_root_mean_squared_error: 1.0344
Epoch 898/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1518 - root_mean_squared_error: 1.0758 - val_loss: 1.1603 - val_root_mean_squared_error: 1.0849
Epoch 899/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1503 - root_mean_squared_error: 1.0802 - val_loss: 1.1709 - val

17/17 [==============================] - 0s 10ms/step - loss: 1.1547 - root_mean_squared_error: 1.0778 - val_loss: 1.1516 - val_root_mean_squared_error: 1.0464
Epoch 989/1000
17/17 [==============================] - 0s 9ms/step - loss: 1.1517 - root_mean_squared_error: 1.0670 - val_loss: 1.1453 - val_root_mean_squared_error: 1.0202
Epoch 990/1000
17/17 [==============================] - 0s 11ms/step - loss: 1.1507 - root_mean_squared_error: 1.0626 - val_loss: 1.1489 - val_root_mean_squared_error: 1.0980
Epoch 991/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1508 - root_mean_squared_error: 1.0473 - val_loss: 1.1536 - val_root_mean_squared_error: 1.0463
Epoch 992/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1549 - root_mean_squared_error: 1.0566 - val_loss: 1.1498 - val_root_mean_squared_error: 1.0305
Epoch 993/1000
17/17 [==============================] - 0s 7ms/step - loss: 1.1451 - root_mean_squared_error: 1.0767 - val_loss: 1.1565 - val_r

Now let's produce an output from the model given the test examples. The output is now a distribution, and we can use its mean and variance to compute the confidence intervals (CI) of the prediction.

In [32]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 6.16, stddev: 0.79, 95% CI: [7.7 - 4.62] - Actual: 6.0
Prediction mean: 6.16, stddev: 0.74, 95% CI: [7.61 - 4.7] - Actual: 6.0
Prediction mean: 6.55, stddev: 0.83, 95% CI: [8.19 - 4.92] - Actual: 7.0
Prediction mean: 5.16, stddev: 0.67, 95% CI: [6.48 - 3.84] - Actual: 5.0
Prediction mean: 5.51, stddev: 0.69, 95% CI: [6.86 - 4.16] - Actual: 5.0
Prediction mean: 5.94, stddev: 0.74, 95% CI: [7.39 - 4.5] - Actual: 6.0
Prediction mean: 5.77, stddev: 0.73, 95% CI: [7.21 - 4.34] - Actual: 6.0
Prediction mean: 5.82, stddev: 0.73, 95% CI: [7.26 - 4.38] - Actual: 7.0
Prediction mean: 5.89, stddev: 0.76, 95% CI: [7.37 - 4.4] - Actual: 8.0
Prediction mean: 5.99, stddev: 0.77, 95% CI: [7.5 - 4.47] - Actual: 5.0
